In [1]:
import os.path
import re
import pprint as pp
import yaml
import pathlib

import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.backends.backend_pdf import PdfPages
import scipy.signal
from pymongo import MongoClient
from gridfs import GridFS

import eim

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
song_label = 'S002'
song_title_for_filename = 'hallelujah'
critical_time = 57.23

## Setup connection to the database

In [3]:
creds_path = pathlib.PosixPath('../eim-creds.yml')
with open(creds_path, 'r') as stream:
    try:
        creds = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [4]:
client = MongoClient()
client = MongoClient(host='eim-data.musicsensorsemotion.com', 
                     port=27017, 
                     username=creds['username'], 
                     password=creds['password'], 
                     authSource='admin', 
                     ssl=True)
db = client.eim

In [5]:
trials = db.trials
media = db.media
signals = db.signals

In [6]:
song_media = media.find_one({'label': song_label})
pp.pprint(song_media)

{'_id': ObjectId('538b777e2212e1eda2ff48bf'),
 'artist': 'Jeff Buckley',
 'bpm': None,
 'comments': None,
 'emotion_tags': [ObjectId('538bd9352212e1eda2ff529a')],
 'excerpt_end_time': 410.43,
 'excerpt_start_time': 300.27,
 'file': ObjectId('538b8b6a2bfb12c72c651e86'),
 'genres': [ObjectId('538bd1e52212e1eda2ff5297')],
 'has_lyrics': True,
 'key': None,
 'label': 'S002',
 'source': None,
 'title': 'Hallelujah',
 'type': 'audio',
 'year': datetime.datetime(1994, 1, 1, 0, 0)}


In [7]:
control_media = media.find_one({'label': 'C002'})
pp.pprint(control_media)

{'_id': ObjectId('56ccaaa8d4c6ac453fb18ec7'),
 'artist': None,
 'bpm': None,
 'comments': '0m30s of pink noise used as a control sound in several '
             'experiments',
 'emotion_tags': [],
 'excerpt_end_time': 0.0,
 'excerpt_start_time': 30.015,
 'file': ObjectId('56cca64ed4c6ac453fb189cc'),
 'genres': [],
 'has_lyrics': False,
 'key': None,
 'label': 'C002',
 'source': None,
 'title': 'Pink Noise',
 'type': 'audio',
 'year': None}


In [8]:
song_media_id, control_media_id = song_media['_id'], control_media['_id']
song_media_id, control_media_id

(ObjectId('538b777e2212e1eda2ff48bf'), ObjectId('56ccaaa8d4c6ac453fb18ec7'))

## Find all trials signals that used target stimulus

Save those `trials` and `signals` with better than 80% quality.

In [9]:
all_stimulus_signals = signals.find({'label': song_label})

# Get all trials that have a good (> 0.8) quality score for this stimulus
good_stimulus_trials_and_signals = []
for signal in all_stimulus_signals:
    if 'derived_eda_data_file' not in signal.keys():
        continue
    data_file_id = signal['derived_eda_data_file']
    data_file = db.signals.files.find_one({'_id': data_file_id})
    if 'metadata' in data_file.keys():
        if 'quality' in data_file['metadata'].keys():
            quality = data_file['metadata']['quality']
            if quality > 0.8 and 'trial' in signal.keys() and signal['trial'] is not None:
                instance_dict = {
                    'trial': signal['trial'],
                    'signal': signal,
                    'data_file': data_file,
                    'quality': quality
                }
                good_stimulus_trials_and_signals.append(instance_dict)

Creater a helper function to add a visual indication of the "reaction range".

In [10]:
def add_reaction_indicator(target_time, lead_in, lead_out):    
    plt.axvline(target_time, color='k', linestyle='dashed', linewidth=1, alpha=0.5)
    ax = plt.gca()
    xlims = ax.get_xlim()
    ylims = ax.get_ylim()
    left_rectangle = patches.Rectangle((xlims[0], ylims[0]), target_time - lead_in - xlims[0], ylims[1] - ylims[0])
    right_rectangle = patches.Rectangle((target_time + 5., ylims[0]), xlims[1] - (target_time + lead_out), ylims[1] - ylims[0])
    left_rectangle.set_alpha(0.2)
    right_rectangle.set_alpha(0.2)
    ax.add_patch(left_rectangle)
    ax.add_patch(right_rectangle)

## "Reaction finder" helper function

In [11]:
def reaction_present(timestamps, eda, stimulus_time, lead_in, lead_out, height, distance, prominence, width, plot=False):
    
    # Detrend EDA signal
    eda_detrended = scipy.signal.detrend(eda)
    
    # Find peaks in EDA signal
    fs = 1. / np.mean(scipy.diff(timestamps))
    peak_indices, _ = scipy.signal.find_peaks(eda_detrended, height=height, distance=distance, prominence=prominence, width=width)
    peak_timestamps = peak_indices / fs
    
    peaks_in_window = peak_timestamps[(peak_timestamps >= stimulus_time - lead_in) & (peak_timestamps <= stimulus_time + lead_out)]
    
    if plot:
        fig = plt.figure(figsize=(16, 8))
        plt.plot(df.adjusted_time, df.eda_detrended)
        plt.scatter(df.adjusted_time[peak_indices], df.eda_detrended[peak_indices], c='r', marker='x')
        add_reaction_indicator(hallelujah_time, lead_in, lead_out)
        plt.title('find_peaks with minimum height = None, minimum distance = 1s, minimum prominence = 0.2, and minimum width = 0.9s');
    
    return len(peaks_in_window) > 0

In [12]:
def indicators_from_categories(dataframe, prefix=''):
    """Converts a DataFrame of category columns to indicator columns

    For instance, given a DataFrame that looks something like the following:

    var1 | var2 | var3
    ------------------
    foo  | bar  |
    foo  | bar  | baz
    baz  |      |

    This function returns the following:

    foo | bar | baz
    ---------------
    1   | 0   | 0
    1   | 1   | 1
    0   | 0   | 1

    >>> df = pd.DataFrame(
    ...     data={
    ...         'var1': ['foo','foo','baz'],
    ...         'var2': ['bar', 'bar', None],
    ...         'var3': [None, 'baz', None]
    ...     }
    ... )
    >>> indicated = indicators_from_categories(df, '')
    >>> np.testing.assert_equal(indicated.foo, np.array([1, 1, 0]))
    >>> np.testing.assert_equal(indicated.bar, np.array([1, 1, 0]))
    >>> np.testing.assert_equal(indicated.baz, np.array([0, 1, 1]))
    """
    old_df = dataframe.copy()
    new_df = dataframe.copy()

    # First, get the unique column names
    var_list = np.array([])
    for column in old_df.columns:
        var_list = np.append(var_list, old_df[column].dropna().unique())
    var_list = [prefix + v for v in set(var_list)]

    # Add these columns
    for var in var_list:
        new_df[var] = pd.Series(np.zeros(new_df.shape[0]), index=old_df.index, dtype=np.int32)

    # Iterate each row and populate new columns
    for ix in old_df.index:
        row = old_df.loc[ix]
        for style in old_df.loc[ix].values:
            new_df.loc[ix, prefix + str(style)] = 1

    return new_df[var_list]

### Process good signals

Check each signal for a reaction, and store the result in the dataframe.

In [13]:
lead_in, lead_out = (4., 6.)

file_system = GridFS(db, collection='signals')

no_reaction_signal_list = []
reaction_signal_list = []

with PdfPages(os.path.join('plots', song_title_for_filename + '_no_reaction.pdf')) as no_reaction_pdf:
    with PdfPages(os.path.join('plots', song_title_for_filename + '_reaction.pdf')) as reaction_pdf:
        for instance_dict in good_stimulus_trials_and_signals:
            data_file = instance_dict['data_file']
            data = file_system.find_one({'filename': data_file['filename']})
            df = pd.read_csv(data, usecols=['adjusted_time', 'eda_cleaned'])

            df['eda_detrended'] = scipy.signal.detrend(df.eda_cleaned)

            fs = 1. / np.mean(scipy.diff(df.adjusted_time))

            min_peak_height = None
            min_peak_distance = fs * 1.
            min_peak_prominence = df['eda_detrended'].std() * 1.5 # 1.4043483998165829
            min_peak_width = fs * 0.9

            reaction = reaction_present(df.adjusted_time, df.eda_cleaned, critical_time, lead_in, lead_out, min_peak_height, min_peak_distance, min_peak_prominence, min_peak_width)

            if reaction:
                plt.plot(df.adjusted_time, df.eda_detrended)
                plt.xlabel('Time [seconds]')
                plt.ylabel('Skin conductance (detrended) [$\mu$S]')
                plt.title('Example with reaction ({})'.format(instance_dict['trial']))
                add_reaction_indicator(critical_time, lead_in, lead_out)
                plt.tight_layout()
                reaction_pdf.savefig()
                plt.close()
                reaction_signal_list.append(df[['adjusted_time', 'eda_detrended']])
            else:
                plt.plot(df.adjusted_time, df.eda_detrended)
                plt.xlabel('Time [seconds]')
                plt.ylabel('Skin conductance (detrended) [$\mu$S]')
                plt.title('Example without reaction ({})'.format(instance_dict['trial']))
                add_reaction_indicator(critical_time, lead_in, lead_out)
                plt.tight_layout()
                no_reaction_pdf.savefig()
                plt.close()
                no_reaction_signal_list.append(df[['adjusted_time', 'eda_detrended']])

            instance_dict['reaction'] = reaction
            instance_dict['df'] = df

Plot the results of the above processing.

In [16]:
fig, [no_ax, yes_ax] = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(18,12))

for df in no_reaction_signal_list[0:15]:
    no_ax.plot(df.adjusted_time, df.eda_detrended)

for df in reaction_signal_list[0:15]:
    yes_ax.plot(df.adjusted_time, df.eda_detrended)

no_ax.set_ylim([-5, 5])
    
plt.sca(no_ax)
add_reaction_indicator(critical_time, lead_in, lead_out)
plt.sca(yes_ax)
add_reaction_indicator(critical_time, lead_in, lead_out)

no_ax.set_title('No reaction presented (n={})'.format(len(no_reaction_signal_list)))
yes_ax.set_title('Reaction presented (n={})'.format(len(reaction_signal_list)));

Helper function to pull in associated data for a given `trial`

In [17]:
def associate_trial(trial, eim_db):
    # Copy input
    associated_trial = dict(trial)
    
    # Get associated experiments, media, and signals
#     associated_trial['experiment'] = eim_db.experiments.find_one({"_id": trial['experiment']})
#     associated_trial['media'] = [eim_db.media.find_one({"_id": _id}) for _id in trial['media']]
#     associated_trial['signals'] = [eim_db.signals.find_one({"_id": _id}, {"metadata": 0}) for _id in trial['signals']]
    
    return associated_trial

Helper function to "flatten" a `trial`

In [18]:
def flatten_trial(trial, delimiter='.'):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + delimiter)
        elif type(x) is list:
            if len(x) == 0:
                out[name[:-1]] = None
            else:
                i = 0
                for a in x:
                    flatten(a, name + str(i) + delimiter)
                    i += 1
        else:
            out[name[:-1]] = x

    flatten(trial)
    return out

## Collect all trials together, associate related entities, and put them in a DataFrame

In [19]:
def media_index(trial, media_id):
    if media_id in trial['media']:
        return trial['media'].index(media_id)
    else:
        return None

In [20]:
def extract_ratings_data(trial, index, prefix=''):
    all_ratings = trial['answers']['ratings']
    if all_ratings is not None:
        for key in all_ratings.keys():
            if index is not None:
                trial[prefix + key] = all_ratings[key][index]
            else:
                trial[prefix + key] = None
    
    return trial

In [21]:
def remove_bad_keys(trial):
    try:
        trial['metadata'].pop('terminal')
    except:
        pass
    
    try:
        trial['metadata'].pop('session_number')
    except:
        pass
        
    try:
        trial['answers'].pop('most_engaged')
    except:
        pass
        
    try:
        trial['answers'].pop('most_enjoyed')
    except:
        pass
        
    try:
        trial['answers'].pop('ratings')
    except:
        pass
        
    try:
        trial['answers'].pop('emotion_indices')
    except:
        pass
        
    try:
        trial.pop('media')
    except:
        pass
        
    try:
        trial.pop('date')
    except:
        pass
        
    try:
        trial.pop('valid')
    except:
        pass
        
    try:
        trial.pop('random')
    except:
        pass
        
    try:
        trial.pop('experiment')
    except:
        pass
        
    try:
        trial.pop('signals')
    except:
        pass
        
    try:
        trial.pop('timestamps')
    except:
        pass

    return trial

In [24]:
all_trials = pd.DataFrame()
all_trials_n = len(good_stimulus_trials_and_signals)
for n, instance_dict in enumerate(good_stimulus_trials_and_signals):
    print('\rProcessing trial {} of {}'.format(n + 1, all_trials_n), end='')
    t = db.trials.find_one({'_id': instance_dict['trial']})
    t['critical_reaction'] = instance_dict['reaction']

    # Pull out experimental and control stimuli data
    ci, si = media_index(t, control_media_id), media_index(t, song_media_id)
    
    try:
        t = extract_ratings_data(t, ci, 'control_')
        t = extract_ratings_data(t, si, 'song_')
    except:
        continue
    
    # Remove unwanted keys
    t = remove_bad_keys(t)
    
    t = dict(flatten_trial(t))
    
    # Create a dataframe from this trial and add to the master dataframe
    df = pd.DataFrame(data=t, index=[t['_id']])
    all_trials = all_trials.append(df, sort=False)

all_trials.loc[:, 'song'] = song_title_for_filename

Processing trial 404 of 404

In [25]:
all_trials.head()

_id metadata.location  \
5410f69308ad6ee3090e387a  5410f69308ad6ee3090e387a            dublin   
5410f69508ad6ee3090e3892  5410f69508ad6ee3090e3892            dublin   
5410f6d008ad6ee3090e3994  5410f6d008ad6ee3090e3994            dublin   
5410f6d008ad6ee3090e3998  5410f6d008ad6ee3090e3998            dublin   
5410f6d008ad6ee3090e399c  5410f6d008ad6ee3090e399c            dublin   

                         answers.music_styles.0 answers.sex answers.dob  \
5410f69308ad6ee3090e387a                   jazz      female  1970-01-01   
5410f69508ad6ee3090e3892                   rock        male  1970-01-01   
5410f6d008ad6ee3090e3994                   rock        male  1986-01-01   
5410f6d008ad6ee3090e3998                   rock      female  1990-01-01   
5410f6d008ad6ee3090e399c                   rock        male  1991-01-01   

                         answers.musical_background answers.nationality  \
5410f69308ad6ee3090e387a                       None               other   
5410f69508ad6ee3090e3892                       None               irish   
5410f6d008ad6ee3090e3994                       None               other   
5410f6d008ad6ee3090e3998                       None               irish   
5410f6d008ad6ee3090e399c                       None               irish   

                         answers.visual_impairments answers.musical_expertise  \
5410f69308ad6ee3090e387a                       None                      None   
5410f69508ad6ee3090e3892                       None                      None   
5410f6d008ad6ee3090e3994                       None                      None   
5410f6d008ad6ee3090e3998                       None                      None   
5410f6d008ad6ee3090e399c                       None                      None   

                          answers.hearing_impairments     ...      \
5410f69308ad6ee3090e387a                        False     ...       
5410f69508ad6ee3090e3892                        False     ...       
5410f6d008ad6ee3090e3994                        False     ...       
5410f6d008ad6ee3090e3998                        False     ...       
5410f6d008ad6ee3090e399c                        False     ...       

                          answers.personality.imagination  \
5410f69308ad6ee3090e387a                              NaN   
5410f69508ad6ee3090e3892                              NaN   
5410f6d008ad6ee3090e3994                              NaN   
5410f6d008ad6ee3090e3998                              NaN   
5410f6d008ad6ee3090e399c                              NaN   

                          answers.personality.reserved  \
5410f69308ad6ee3090e387a                           NaN   
5410f69508ad6ee3090e3892                           NaN   
5410f6d008ad6ee3090e3994                           NaN   
5410f6d008ad6ee3090e3998                           NaN   
5410f6d008ad6ee3090e399c                           NaN   

                         answers.personality.outgoing  \
5410f69308ad6ee3090e387a                          NaN   
5410f69508ad6ee3090e3892                          NaN   
5410f6d008ad6ee3090e3994                          NaN   
5410f6d008ad6ee3090e3998                          NaN   
5410f6d008ad6ee3090e399c                          NaN   

                         answers.personality.lazy answers.personality.stress  \
5410f69308ad6ee3090e387a                      NaN                        NaN   
5410f69508ad6ee3090e3892                      NaN                        NaN   
5410f6d008ad6ee3090e3994                      NaN                        NaN   
5410f6d008ad6ee3090e3998                      NaN                        NaN   
5410f6d008ad6ee3090e399c                      NaN                        NaN   

                         answers.personality.nervous  \
5410f69308ad6ee3090e387a                         NaN   
5410f69508ad6ee3090e3892                         NaN   
5410f6d008ad6ee3090e3994                         NaN   
5410f6d008ad6ee3090e3998                      

In [26]:
music_styles_columns = [x for x in all_trials.columns if re.search('music_styles', x)]
music_styles_df = all_trials[music_styles_columns]
music_styles_df.columns

Index(['answers.music_styles.0', 'answers.music_styles.1',
       'answers.music_styles.2', 'answers.music_styles.3',
       'answers.music_styles.4', 'answers.music_styles.5',
       'answers.music_styles.6', 'answers.music_styles.7'],
      dtype='object')

In [27]:
new_styles_df = indicators_from_categories(music_styles_df, prefix='answers.musicpref.')

In [28]:
new_styles_df.head()

answers.musicpref.traditional_irish  \
5410f69308ad6ee3090e387a                                    0   
5410f69508ad6ee3090e3892                                    0   
5410f6d008ad6ee3090e3994                                    0   
5410f6d008ad6ee3090e3998                                    0   
5410f6d008ad6ee3090e399c                                    1   

                          answers.musicpref.jazz  answers.musicpref.folk  \
5410f69308ad6ee3090e387a                       1                       0   
5410f69508ad6ee3090e3892                       0                       0   
5410f6d008ad6ee3090e3994                       0                       0   
5410f6d008ad6ee3090e3998                       0                       0   
5410f6d008ad6ee3090e399c                       1                       0   

                          answers.musicpref.hiphop  \
5410f69308ad6ee3090e387a                         0   
5410f69508ad6ee3090e3892                         0   
5410f6d008ad6ee3090e3994                         0   
5410f6d008ad6ee3090e3998                         0   
5410f6d008ad6ee3090e399c                         0   

                          answers.musicpref.classical  answers.musicpref.none  \
5410f69308ad6ee3090e387a                            0                       0   
5410f69508ad6ee3090e3892                            0                       0   
5410f6d008ad6ee3090e3994                            0                       0   
5410f6d008ad6ee3090e3998                            0                       0   
5410f6d008ad6ee3090e399c                            0                       0   

                          answers.musicpref.rock  answers.musicpref.hip_hop  \
5410f69308ad6ee3090e387a                       0                          0   
5410f69508ad6ee3090e3892                       1                          0   
5410f6d008ad6ee3090e3994                       1                          0   
5410f6d008ad6ee3090e3998                       1                          0   
5410f6d008ad6ee3090e399c                       1                          0   

                          answers.musicpref.world  answers.musicpref.pop  \
5410f69308ad6ee3090e387a                        0                      0   
5410f69508ad6ee3090e3892                        0                      1   
5410f6d008ad6ee3090e3994                        1                      1   
5410f6d008ad6ee3090e3998                        0                      1   
5410f6d008ad6ee3090e399c                        0                      1   

                          answers.musicpref.dance  
5410f69308ad6ee3090e387a                        0  
5410f69508ad6ee3090e3892                        1  
5410f6d008ad6ee3090e3994                        0  
5410f6d008ad6ee3090e3998                        1  
5410f6d008ad6ee3090e399c                        1

In [29]:
all_trials = all_trials.drop(columns=music_styles_columns)
new_df = pd.concat([all_trials, new_styles_df], axis=1, sort=False)
csv_path = os.path.join('data', 'good_' + song_title_for_filename + '_trials_and_signals.csv')
new_df.to_csv(csv_path)
print('CSV file written to: {}'.format(csv_path))

CSV file written to: data/good_hallelujah_trials_and_signals.csv
